#Setup

In [ ]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31

# Beam: typehints

In [ ]:
# importowanie modułu apache_beam
import apache_beam as beam

# Tworzenie potoku Beam
p = beam.Pipeline()

# Definicja klasy FilterEvensDoFn dziedziczącej po beam.DoFn, która przetwarza elementy
# @beam.typehints.with_input_types(int) - Dekorator wskazujący, że wejściowe elementy tej klasy mają być typu int
@beam.typehints.with_input_types(int)
class FilterEvensDoFn(beam.DoFn):
    # Metoda process, która jest wywoływana dla każdego elementu w strumieniu danych
    def process(self, element):
        # Sprawdzanie, czy element jest liczbą parzystą
        if element % 2 == 0:
            # Jeśli jest parzysty, element jest przekazywany do strumienia wynikowego
            yield element

# Tworzenie potoku danych:
evens = ( p
         # beam.Create - tworzy PCollection z listy ['1', '2', '3'], ale elementy są typu string
         | beam.Create(['1', '2', '3'])
         # beam.ParDo - stosuje transformację FilterEvensDoFn, która filtruje tylko liczby parzyste
         | beam.ParDo(FilterEvensDoFn())
        )

# Uruchomienie potoku Beam, aby przetworzyć dane
p.run()

In [ ]:
import apache_beam as beam  # Importujemy Apache Beam, który służy do przetwarzania danych w potokach

# Tworzymy obiekt potoku Beam, który będzie przechodził przez różne transformacje
p = beam.Pipeline()

# Definiujemy transformację na danych wejściowych
evens = (
    p
    | beam.Create(['one', 'two', 'three'])  # Tworzymy PCollection z trzema wartościami typu string
    | beam.Filter(lambda x: x % 2 == 0).with_input_types(int)  # Próbujemy odfiltrować liczby parzyste, ale mamy błąd!
)

# Uruchamiamy potok, aby wykonać transformacje
p.run()

#Beam: type_hints_with_custom_data_type

In [ ]:
import apache_beam as beam  # Importujemy Apache Beam do przetwarzania danych
import typing  # Importujemy moduł typing do określania typów danych

# Definiujemy klasę Employee, która będzie naszym niestandardowym typem danych
class Employee(object):
    def __init__(self, id, name):
        self.id = id  # Identyfikator pracownika
        self.name = name  # Imię pracownika

# Tworzymy własnego Codera do serializacji i deserializacji obiektów Employee
class EmployeeCoder(beam.coders.Coder):

    def encode(self, employee):
        """Serializuje obiekt Employee do postaci bajtów"""
        return ('%s:%s' % (employee.id, employee.name)).encode('utf-8')

    def decode(self, s):
        """Deserializuje bajty do obiektu Employee"""
        return Employee(*s.decode('utf-8').split(':'))

    def is_deterministic(self):
        """Zwraca True, co oznacza, że kodowanie jest deterministyczne (powtarzalne)"""
        return True

# Rejestrujemy naszego Codera dla klasy Employee, aby Beam wiedział, jak ją serializować
beam.coders.registry.register_coder(Employee, EmployeeCoder)

# Funkcja do przekształcenia danych wejściowych (wiersza z pliku) w obiekt Employee oraz pensję
def split_file(input):
    name, id, salary = input.split(',')  # Rozdzielamy wiersz na imię, ID i pensję
    return Employee(id, name), int(salary)  # Zwracamy parę (Employee, pensja)

# Tworzymy potok Beam
p = beam.Pipeline()

# Definiujemy przetwarzanie w potoku
result = (
    p
    | beam.io.ReadFromText('data.txt')  # Odczytujemy dane z pliku tekstowego
    | beam.Map(split_file)  # Parsujemy każdą linię na obiekt Employee i pensję
    | beam.CombinePerKey(sum).with_input_types(typing.Tuple[Employee, int])
    # Grupujemy dane według klucza (Employee) i sumujemy wartości (pensje)
)

# Uruchamiamy potok
p.run()